# RAG 테스트

1. 랭체인의 llm chain을 사용하지 않은 경우의 결과를 봅니다.
2. 오픈소스 llm과 GPT api와의 성능 차이를 비교합니다.
2. vLLM을 테스트합니다.

**LLM**: [yanolja/EEVE-Korean-Instruct-10.8B-v1.0](https://huggingface.co/yanolja/EEVE-Korean-Instruct-10.8B-v1.0)
/ gpt-3.5-turbo

**document**: PDFLoader, pdf 업로드하고 실행

**text splitter**: RecursiveCharacterTextSplitter

**embedding**: jhgan/ko-sbert-nli

**vectorDB**: FAISS / InMemoryVectorStore


# 랭체인의 llm chain을 사용하지 않았을때

똑같이 결과가 매우 느리게 나왔다. llm의 입력으로 긴 텍스트를 줄 수록 결과가 매우 느리게 나오는 것 같다.

In [ ]:
%%capture
!pip -q install peft accelerate bitsandbytes langchain pypdf chromadb transformers sentence-transformers faiss-gpu

In [ ]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes accelerate
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_id = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

#model = AutoModelForCausalLM.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")
#tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")

prompt_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nHuman: {prompt}\nAssistant:\n"
text = '한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.\n\n(A) 경성\n(B) 부산\n(C) 평양\n(D) 서울\n(E) 전주'
model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors='pt')

outputs = model.generate(**model_inputs, max_new_tokens=256)
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(output_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2091: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
Human: 한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.

(A) 경성
(B) 부산
(C) 평양
(D) 서울
(E) 전주
Assistant:
(D) 서울이 한국의 수도입니다. 서울은 나라의 북동부에 위치해 있으며, 정치, 경제, 문화의 중심지입니다.


### Document Load, Split, Store, 사용자 질문에 대한 문서 Retreive까지의 과정

In [ ]:
!pip install langchain_community langchain_huggingface

In [ ]:
# # PDF 로더 설정
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
loader = PyPDFLoader("/content/쉬어매드니스감상문.pdf") # 왼쪽 메뉴에서 업로드 아이콘 눌러서 pdf 업로드한 후 파일 이름에 맞게 바꾸기
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

# 임베딩 모델 설정
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cuda'}
# normalize_embeddings: 벡터의 크기를 1로 만들어 코사인 유사도 계산할 때 내적 사용 가능
encode_kwargs = {'normalize_embeddings': True}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
vectorstore = FAISS.from_documents(texts, hf_embeddings)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)
query = "쉬어 매드니스 연극을 추천하는 이유를 알려줘."
docs = vectorstore.similarity_search(query)
context = "\n\n".join([doc.page_content for doc in docs])
context

In [ ]:
docs_and_scores = vectorstore.similarity_search_with_score(query)
docs_and_scores

### 검색한 문서 + 사용자 질문 => LLM => 답변 출력
랭체인 | 연산자 사용 안하고

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

#model = AutoModelForCausalLM.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")
#tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")

def get_llm_output(prompt):
    model_inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(**model_inputs, max_new_tokens=256)
    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return output_text

이것도 10분 이상.. 결과가 나오지 않음

In [ ]:
# Query 및 문서 검색
query = "쉬어 매드니스 연극을 추천하는 이유를 알려줘."
docs = vectorstore.similarity_search(query)

# docs에서 문서 내용을 추출해 context로 사용
context = "\n\n".join([doc.page_content for doc in docs])

# 프롬프트 템플릿 설정
prompt_template = """
당신은 한국어 질문에 답변하는 어시스턴트입니다.
다음의 정보를 참고하여 질문에 답하세요. 모르는 정보라면 모른다고 답하세요.

참고 정보:
{context}

질문:
{question}
"""

# context와 question을 프롬프트에 삽입
final_prompt = prompt_template.format(context=context, question=query)

# LLM으로 답변 생성
result = get_llm_output(final_prompt)

# 결과 출력
print("LLM 답변: ", result)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1934: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


KeyboardInterrupt: 

패키지 버전 저장

In [ ]:
!pip freeze > requirements.txt

In [ ]:
# install 할 때
!pip install -r requirements.txt

In [ ]:
from google.colab import files
files.download('requirements.txt')

# GPT api 사용했을 때의 속도 확인

https://python.langchain.com/docs/tutorials/pdf_qa/

In [1]:
!pip install -qU pypdf langchain_community langchain-openai langchain_huggingface transformers accelerate faiss-gpu
!pip install -U bitsandbytes

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/쉬어매드니스감상문.pdf"
loader = PyPDFLoader(file_path)# 왼쪽 메뉴에서 업로드 아이콘 눌러서 pdf 업로드한 후 파일 이름에 맞게 바꾸기

docs = loader.load()

print(len(docs))

In [3]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

### Runnable로 openai gpt 사용하는 경우 실행

In [4]:
from langchain_openai import ChatOpenAI
OPENAI_API_KEY = ""  # API 키 입력
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)

### Runnable로 huggingface 오픈소스 llm EEVE 사용하는 경우 실행

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

# Quantization config for 4-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"  # Example model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

#model = AutoModelForCausalLM.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")
#tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")

prompt_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nHuman: {prompt}\nAssistant:\n"
text = '한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.\n\n(A) 경성\n(B) 부산\n(C) 평양\n(D) 서울\n(E) 전주'
model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors='pt')

outputs = model.generate(**model_inputs, max_new_tokens=256)
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(output_text)

In [ ]:
# Create Hugging Face pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=100,
)

# Wrap pipeline into LangChain's HuggingFacePipeline for compatibility
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

매우 짧은 아래의 질문에 대해 실행 시간 2분.

추가로 RAG 구조를 이용할거라 상관은 없겠지만 답변이 틀렸다...

In [ ]:
llm.invoke("겨울의 대삼각형에 대해 알려줘.")

답변 10분이상 X

In [ ]:
prompt = """
### [INST]
주어진 정보에 기반해 질문에 한국어로 답하세요.
겨울의 대삼각형(영어: Winter Triangle)은 겨울 밤하늘에 보이는 세 개의 별을 이은 성군으로,[1] 큰개자리의 시리우스, 오리온자리의 베텔게우스, 작은개자리의 프로키온을 이은 천구상의 정삼각형이다.[2]

가시성
겨울의 대삼각형.
겨울의 대삼각형은 북반구 중위도에서 겨울 밤하늘 내내 높이 떠오르며, 가을 새벽에 동쪽에서, 또는 봄 이른 저녁에 서쪽에서도 볼 수 있다. 남반구에서는 여름 동안 볼 수 있으며, 삼각형이 낮게 뜨고 뒤집혀 보인다.[3]
삼각형은 희미한 별자리인 외뿔소자리의 대부분을 감싸며, 대삼각형은 1등성들로만 구성된 데 비해 외뿔소자리는 1등성이 4등급일 정도로 어둡기 때문에 외뿔소자리를 눈으로 식별하기는 쉽지 않다.
삼각형 주변의 밝은 별로는 오리온자리의 리겔, 황소자리의 알데바란, 쌍둥이자리의 폴룩스와 카스토르, 마차부자리의 카펠라가 있다.

여름의 대삼각형(Summer Triangle)은 여름철 북반구 밤하늘에서 쉽게 볼 수 있는 밝은 별 3개가 이루는 가상의 삼각형이다. 백조자리의 데네브, 독수리자리의 알타이르, 거문고자리의 베가로 이뤄지는 삼각형 모양의 성군이다.
상세
이 용어는 1950년대에 미국 작가 한스 아우구스토 레이와 영국 천문학자 패트릭 무어 경이 대중화시켰으며, 1913년에 사용된 별자리 안내서에서도 발견된다.[1] 오스트리아의 천문학자 오즈발트 토마스는 이 별들을 1920년대에 ‘대삼각형’(Grosses Dreieck), 1934년에 ‘여름의 삼각형’(Sommerliches Dreieck)이라고 불렀다. 1866년에는 요제프 요한 폰 리트로우가 자기 성도책에서 거문고자리, 독수리자리, 백조자리를 묶어 다루며 ‘하늘에 떠 있는 이등변삼각형’(gleicheschenkliges Dreieck am Himmel)을 언급하였고,[2] 요한 엘레르트 보데는 1816년의 책에서 명칭을 달아놓지는 않았지만 세 별을 서로 연결해 놓았다.
북반구 중위도에서 여름철 자정 무렵에 여름의 대삼각형은 거의 머리 바로 위에 위치해 있다. 하지만 여름에만 보이는 것은 아니고 봄에는 신새벽에 동쪽에서, 가을에는 저녁에 서쪽에서 11월까지 볼 수 있다. 남반구에서는 겨울철 하늘에 뒤집혀서 보인다.

### 사용자의 질문:
겨울의 대삼각형에 대해 알려줘.

### 답변:
[/INST]
"""
llm.invoke(prompt)

### text splitter, vectorstore, embedding 등

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cuda'}
# normalize_embeddings: 벡터의 크기를 1로 만들어 코사인 유사도 계산할 때 내적 사용 가능
encode_kwargs = {'normalize_embeddings': True}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

### 아래 두 블럭은 두 가지 VectorStore 라이브러리

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=hf_embeddings
)

retriever = vectorstore.as_retriever()

In [ ]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(splits, hf_embeddings)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)
query = "쉬어 매드니스 연극을 추천하는 이유를 알려줘."
docs = vectorstore.similarity_search(query)
context = "\n\n".join([doc.page_content for doc in docs])
context

### 두 가지의 응답 도출 방법

In [ ]:
prompt_template = """
### [INST]
주어진 정보에 기반해 질문에 한국어로 답하세요.
{context}

### 사용자의 질문:
{question}

[/INST]
"""

# Create prompt from prompt template
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

#llm_chain = prompt | llm

# RAG 체인 생성
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

result = rag_chain.invoke("쉬어매드니스 연극을 추천하지 않는 이유가 있다면?")
result

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "당신은 질문에 답변하는 어시스턴트입니다."
    "다음에 제공되는 문맥을 사용하여 질문에 답변하세요. "
    "답을 모르면 모른다고 말하세요."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "쉬어매드니스 연극을 추천하지 않는 이유가 있다면?"})

results

# vLLM

In [ ]:
!pip install torch==2.4.1
!pip install vllm
!pip install bitsandbytes>=0.44.0

In [ ]:
from vllm import LLM
import torch

model_id = "yongsun-shim/eeve-4bit-test"
llm = LLM(
    model=model_id,
    dtype=torch.float16,  # dtype을 float16으로 설정
    trust_remote_code=True,
    quantization="bitsandbytes",
    load_format="bitsandbytes"
)

In [ ]:
outputs = llm.generate("겨울철의 대삼각형에 대해 알려줘.")
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")